In [27]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("topcv").getOrCreate()

24/06/30 15:55:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [28]:
# Define the path to your JSON file
from pyspark.sql.functions import col, struct, regexp_extract, udf, date_sub, to_date, when, regexp_extract, expr, explode, col, regexp_replace, to_timestamp, concat, lit, trim, date_format
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, ArrayType
import json
import re
import pymongo
json_file_path = 'topcvnew.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)

# df_exploded = df.select(explode(col("category")).alias("unique_category"))
# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     # pass
#     print(row.unique_category)
    # parts = [part.strip() for part in row.unique_category.split(',')]
    # print(parts)

# df_exploded = df.select(explode(col("category")).alias("unique_category"))

# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     pass
#     print(row.unique_category)
# unique_experience_df = df.select("location").distinct()

# unique_experience_df.show()
# from pyspark.sql.functions import col
# unique_values = df.select(col("category")).distinct().collect()
# In ra các giá trị duy nhất
# for row in unique_values:
#     if row.category is None:
#         print("None ne")
#     print(row.category)


# Salary


In [29]:
df = spark.read.option("multiline","true").json(json_file_path)
df.dropDuplicates()
df = df.filter(col("company").isNotNull())
date_pattern = r'(\d{2}/\d{2}/\d{4})'

df = df.withColumn("certificate", lit(0))
# df = df.withColumn("age", struct(lit(0).alias("type")))
df = df.withColumn(
    "sex",
     when(col("sex") == "Nam", 1)
    .when(col("sex") == "Nữ", 2)
    .otherwise(0)
)

# Experience
def parse_experience(experience):
    if experience is None or experience == "Không yêu cầu kinh nghiệm":
        return { "type": 0 }
    if "-" in experience:
        strip_experience = experience.replace("Năm", "").strip()
        min_exp, max_exp = strip_experience.split("-")
        return {"type": 1, "min": int(min_exp), "max": int(max_exp) }
    elif "Dưới" in experience:
        strip_experience = experience.replace("Dưới", "").replace("năm", "").strip()
        max = strip_experience
        return { "type": 3, "max": int(max) }
    elif "Trên" in experience:
        strip_experience = experience.replace("Trên", "").replace("năm", "").strip()
        min = strip_experience
        return { "type": 4, "min": int(min) }
    else:
        fixed = experience.replace("năm", "").strip()
        return { "type": 2, "fixed": int(fixed)}
    return { "type": 0 }
# Type
def get_type_job(type):
    if type == "Toàn thời gian":
        return 0
    elif type == "Bán thời gian":
        return 2
    elif type == "Thực tập":
        return 4
    else:
        return 5

def classify_role(role):
    if role is None:
        return 0
    role = role.strip()
    if role == "Nhân viên":
        return 0
    elif role == "Quản lý / Giám sát" or role == "Trưởng chi nhánh":
        return 1
    elif role == "Cộng tác viên":
        return 6
    elif role == "Trưởng nhóm" or role == "Trưởng/Phó phòng" :
        return 5
    elif role == "Thực tập sinh":
        return 4
    elif role == "Giám đốc" or role == "Tổng giám đốc":
        return 2
    elif role == "Phó giám đốc":
        return 3
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

# Salary
def parse_salary(salary):
    vnd_per_usd = 25
    if salary is None:
        return {"type": 0 }
    elif (salary.strip() == "Thoả thuận"):
        return {"type": 3 }
    elif "triệu" in salary:
            salary = salary.replace(" triệu", "").replace(",", ".").strip()
            if "-" in salary:
                min_salary, max_salary = salary.split("-")
                return {"type": 1, "min": float(min_salary.strip()), "max": float(max_salary.strip()) }
            elif "Tới" in salary:
                min_salary = salary.replace("Tới", "").strip()
                return {"type": 5, "min": float(min_salary)}
            elif "Trên" in salary:
                max_salary = salary.replace("Trên", "").strip()
                return {"type": 4, "max": float(max_salary)}
    elif "USD" in salary:
        salary = salary.replace(" USD", "").replace(",", "").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return {"type": 1, "min": float(min_salary)* vnd_per_usd / 1000, "max": float(max_salary)* vnd_per_usd /1000}
        elif "Tới" in salary:
            max_salary = salary.replace("Tới", "").strip()
            return {"type": 4, "max": float(max_salary)* vnd_per_usd / 1000}
        elif "Trên" in salary:
            min_salary = salary.replace("Trên", "").strip()
            return {"type": 5, "min": float(min_salary)* vnd_per_usd / 1000}
    else:
        return {"type": 0 }
                
# Age
def parse_age(): 
    return { "type": 0 }

# Extract the date string
df = df.withColumn("date_str", regexp_extract(col("expiration"), date_pattern, 1))
# Expiration
df = df.withColumn("expiration", date_format(to_timestamp(concat(col("date_str"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"), "yyyy-MM-dd HH:mm:ss"))

# Update at
hours_pattern = r'(\d+)\s+giờ'
days_pattern = r'(\d+)\s+ngày'
weeks_pattern = r'(\d+)\s+tuần'
# df = df.withColumn("update_time", date_format(to_timestamp(concat(getCrawlTime(col("crawl_time"), col("create_time")), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"), "yyyy-MM-dd HH:mm:ss"))
# Extract the time component and calculate days to subtract
df = df.withColumn("hours", regexp_extract(col("create_time"), hours_pattern, 1).cast("int"))
df = df.withColumn("days", regexp_extract(col("create_time"), days_pattern, 1).cast("int"))
df = df.withColumn("weeks", regexp_extract(col("create_time"), weeks_pattern, 1).cast("int"))

# Calculate the total days to subtract
df = df.withColumn("days_to_subtract", 
                   when(col("hours").isNotNull(), 1)
                   .when(col("days").isNotNull(), col("days"))
                   .when(col("weeks").isNotNull(), col("weeks") * 7)
                   .otherwise(0))

# Convert crawl_time to date
df = df.withColumn("crawl_time", to_date(col("crawl_time"), "dd/MM/yyyy"))

# Calculate update_time by subtracting days_to_subtract from crawl_time
df = df.withColumn("update_time", expr("date_sub(crawl_time, days_to_subtract)"))
df = df.drop(col("create_time"))
df = df.drop(col("crawl_time"))
df = df.drop(col("days_to_subtract"))

# df.select("crawl_time", "create_time", "update_time").show(truncate=False)
def get_list_type_job(types):
    # Thời vụ/ Nghề tự do, Thực tập
    # type = types.split(",")
    if types is None:
        return []
    return [get_type_job(word.strip()) for word in types.split(",")]


classify_type_udf = udf(get_list_type_job, ArrayType(IntegerType()))
df = df.withColumn("type", classify_type_udf(col("type")))

def clean_description(description):
    if description is None:
        return []
    # Filter out empty strings and None values
    cleaned_descriptions = [desc for desc in description if desc is not None and len(desc.strip())> 0]
    # Apply cleaning to filtered descriptions
    cleaned_descriptions = [re.sub(r'^[\-\•\d\.\s]+', '', desc).strip() for desc in cleaned_descriptions]
    return cleaned_descriptions
clean_description_udf = udf(clean_description, ArrayType(StringType()))
df = df.withColumn("description", clean_description_udf(col("description")))
df = df.withColumn("benefit", clean_description_udf(col("benefit")))
df = df.withColumn("requirement", clean_description_udf(col("requirement")))

def getProvince(province):
    if province == "An Giang": 
      location = "AG"
      
    elif province == "Bà Rịa - Vũng Tàu": 
      location = "BV"
      
    elif province == "Bạc Liêu": 
      location = "BL"
      
    elif province == "Bắc Kạn" or province == 'Bắc Cạn': 
      location = "BK"
      
    elif province == "Bắc Giang": 
      location = "BG"
      
    elif province == "Bắc Ninh": 
      location = "BN"
      
    elif province == "Bến Tre": 
      location = "BT"
      
    elif province == "Bình Dương": 
      location = "BD"
      
    elif province == "Bình Định": 
      location = "BDI"
      
    elif province == "Bình Phước": 
      location = "BP"
      
    elif province == "Bình Thuận": 
      location = "BT"
      
    elif province == "Cà Mau": 
      location = "CM"
      
    elif province == "Cao Bằng": 
      location = "CB"
      
    elif province == "Cần Thơ": 
      location = "CT"
      
    elif province == "Đà Nẵng": 
      location = "DN"
      
    elif province == "Đắk Lắk" or province == 'Dak Lak': 
      location = "DL"
      
    elif province == "Đắk Nông" or province == 'Dak Nông': 
      location = "DNO"
      
    elif province == "Điện Biên": 
      location = "DB"
      
    elif province == "Đồng Nai": 
      location = "DNA"
      
    elif province == "Đồng Tháp": 
      location = "DT"
      
    elif province == "Gia Lai": 
      location = "GL"
      
    elif province == "Hà Giang": 
      location = "HG"
      
    elif province == "Hà Nam": 
      location = "HNA"
      
    elif province == "Hà Nội": 
      location = "HN"
      
    elif province == "Hà Tĩnh": 
      location = "HT"
      
    elif province == "Hải Dương": 
      location = "HD"
      
    elif province == "Hải Phòng": 
      location = "HP"
      
    elif province == "Hậu Giang": 
      location = "HGI"
      
    elif province == "Hòa Bình": 
      location = "HB"

    elif province == 'Hồ Chí Minh' or province == "TP.HCM" : 
      location = "HCM"
      
    elif province == "Hưng Yên": 
      location = "HY"
      
    elif province == "Khánh Hòa": 
      location = "KH"
      
    elif province == "Kiên Giang": 
      location = "KG"
      
    elif province == "Kon Tum": 
      location = "KT"
      
    elif province == "Lai Châu": 
      location = "LC"
      
    elif province == "Lạng Sơn": 
      location = "LS"
      
    elif province == "Lào Cai": 
      location = "LCA"
      
    elif province == "Lâm Đồng": 
      location = "LD"
      
    elif province == "Long An": 
      location = "LA"
      
    elif province == "Nam Định": 
      location = "ND"
      
    elif province == "Nghệ An": 
      location = "NA"
      
    elif province == "Ninh Bình": 
      location = "NB"
      
    elif province == "Ninh Thuận": 
      location = "NT"
      
    elif province == "Phú Thọ": 
      location = "PT"
      
    elif province == "Phú Yên": 
      location = "PY"
      
    elif province == "Quảng Bình": 
      location = "QB"
      
    elif province == "Quảng Nam": 
      location = "QNA"
      
    elif province == "Quảng Ngãi": 
      location = "QNG"
      
    elif province == "Quảng Ninh": 
      location = "QN"
      
    elif province == "Quảng Trị": 
      location = "QT"
      
    elif province == "Sóc Trăng": 
      location = "ST"
      
    elif province == "Sơn La": 
      location = "SL"
      
    elif province == "Tây Ninh": 
      location = "TN"
      
    elif province == "Thái Bình": 
      location = "TB"
      
    elif province == "Thái Nguyên": 
      location = "TNG"
      
    elif province == "Thanh Hóa": 
      location = "TH"
      
    elif province == "Thừa Thiên- Huế" or province == "Thừa Thiên Huế": 
      location = "TTH"
      
    elif province == "Tiền Giang": 
      location = "TG"
      
    elif province == "Trà Vinh": 
      location = "TV"
      
    elif province == "Tuyên Quang": 
      location = "TQ"
      
    elif province == "Vĩnh Long": 
      location = "VL"
      
    elif province == "Vĩnh Phúc": 
      location = "VP"
      
    elif province == "Yên Bái": 
      location = "YB"
    else:
        print("No match:\""+ province+ "\"")
        location = 'other'
    return location

# location
def normalize_location(locations):
    arr = []
    province_pattern = r'- ([^:]+):'
    if locations == None:
        return []
    for location in locations:
        province = regexp_extract(location, province_pattern, 1)
        list_add = split(location, ":")
        if len(list_add) > 2:
            address = split(location, ":")[1].trim()
            arr.append({"province": getProvince(province), "district": None, "address": address})
        else: 
            arr.append({"address": None, "district": None, "province": getProvince(province)})
    return arr

category_mapping = {
    "Du lịch": "dv",
    "Hàng gia dụng": "kd",
    "Địa chất / Khoáng sản": "cn_sx",
    "Sản phẩm công nghiệp": "cn_sx",
    "Hành chính / Văn phòng": "hc_ql",
    "In ấn / Xuất bản": "xb",
    "An toàn lao động": "at_an",
    "Spa / Làm đẹp": "dv",
    "Công nghệ cao": "kh_kt",
    "IT Phần cứng / Mạng": "it",
    "Luật/Pháp lý": "law",
    "Marketing / Truyền thông / Quảng cáo": "tt_mkt",
    "Dầu khí/Hóa chất": "cn_sx",
    "Quản lý điều hành": "hc_ql",
    "Thực phẩm / Đồ uống": "tp",
    "Kiến trúc": "tk_kt_nt",
    "Ngành nghề khác": "other",
    "Thư ký / Trợ lý": "hc_ql",
    "Điện / Điện tử / Điện lạnh": "kh_kt",
    "Kinh doanh / Bán hàng": "kd",
    "Bảo trì / Sửa chữa": "kh_kt",
    "Chứng khoán / Vàng / Ngoại tệ": "tc_kt",
    "Hàng không": "vt",
    "Môi trường / Xử lý chất thải": "cn_sx",
    "Bất động sản": "bds",
    "Tư vấn": "dv",
    "Dịch vụ khách hàng": "dv",
    "Y tế / Chăm sóc sức khỏe": "yt_sk",
    "Mỹ phẩm / Trang sức": "kd",
    "Cơ khí / Chế tạo / Tự động hóa": "kh_kt",
    "Giáo dục / Đào tạo": "gd_dt",
    "Hàng tiêu dùng": "kd",
    "Thiết kế đồ họa": "tk_kt_nt",
    "Bán lẻ / Bán sỉ": "kd",
    "Tài chính / Đầu tư": "tc_kt",
    "IT phần mềm": "it",
    "Sản xuất": "cn_sx",
    "Mỹ thuật / Nghệ thuật / Điện ảnh": "tk_kt_nt",
    "Tổ chức sự kiện / Quà tặng": "dv",
    "Hoạch định/Dự án": "xd",
    "Bán lẻ / bán sỉ": "kd",
    "Kế toán / Kiểm toán": "tc_kt",
    "Xuất nhập khẩu": "vt",
    "Công nghệ Ô tô": "cn_sx",
    "Quản lý chất lượng (QA/QC)": "hc_ql",
    "Biên / Phiên dịch": "dv",
    "Thời trang": "tk_kt_nt",
    "Nông / Lâm / Ngư nghiệp": "nn_ln_ts",
    "Điện tử viễn thông": "dv",
    "Ngân hàng / Tài chính": "tc_kt",
    "Công nghệ thông tin": "it",
    "Nhân sự": "hc_ql",
    "Hàng hải": "vt",
    "Y tế / Dược": "yt_sk",
    "Xây dựng": "xd",
    "Dược phẩm / Công nghệ sinh học": "kh_kt",
    "Bảo hiểm": "dv",
    "Việc làm IT": "it",
    "Khách sạn / Nhà hàng": "dv",
    "Thiết kế nội thất": "tk_kt_nt",
    "Logistics": "vt",
    "Bưu chính - Viễn thông": "dv",
    "Hàng cao cấp": "kd",
    "Vận tải / Kho vận": "vt",
    "Dệt may / Da giày": "cn_sx",
    "Hoá học / Sinh học": "kh_kt",
    "Bán hàng kỹ thuật": "kd",
    "Báo chí / Truyền hình": "tt_mkt"
}

def parse_category(categories):
    if categories is None or len(categories) == 0:
        return []
    else :
        category_arr = []
        for category in categories:
            if category in category_mapping:
                category_arr.append(category_mapping[category])
            else:
                print("No result field: "+ category)
                category_arr.append("other")
        return category_arr

df = df.drop(col("province"))
df = df.drop(col("hours"))
df = df.drop(col("days"))
df = df.drop(col("weeks"))
df = df.drop(col("date_str"))

rdd = df.rdd.map(lambda row: {
    **row.asDict(),
    "salary": parse_salary(row["salary"]),
    "age": parse_age(),
    "location": normalize_location(row["location"]),
    "experience": parse_experience(row["experience"]),
    "field": parse_category(row["category"])
})
# df.select('expiration').show(5)
# print(rdd)
# df.printSchema()

# df.select(["role", "url"]).show(30, False)
# df.select("location").show(1, False)

# rdd.top(5)
# rdd.count()

# Insert dữ liệu vào collection "jobs"
def insert_into_mongodb(partition):
    client = pymongo.MongoClient("mongodb://localhost:27017/", username='admin', password='20194856')
    db = client["thesis"]
    collection = db["jobs"]

    for record in partition:
        # print(record)
        collection.insert_one(record)

# # Insert dữ liệu vào collection "jobs"
rdd.foreachPartition(insert_into_mongodb)
# spark.stop()

24/06/30 15:55:59 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)/ 1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  [Previous line repeated 1 more time]
  File "/opt/spark/python/pyspark/rdd.py", line 840, in func
    return f(iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 1795, in func
    r = f(it)
  File "/tmp/ipykernel_5334/2027065263.py", line 485

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (192.168.1.14 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  [Previous line repeated 1 more ti